In [1]:
import bagpipes as pipes
import numpy as np
import itertools
import pickle

filter_list_splus = ['filters/F0378.dat',
                     'filters/F0395.dat',
                     'filters/F0410.dat',
                     'filters/F0430.dat',
                     'filters/F0515.dat',
                     'filters/F0660.dat',
                     'filters/F0861.dat',
                     'filters/uJAVA.dat',
                     'filters/gSDSS.dat',
                     'filters/rSDSS.dat',
                     'filters/iSDSS.dat',
                     'filters/zSDSS.dat']

filter_list_splus_broad = ['filters/uJAVA.dat',
                           'filters/gSDSS.dat',
                           'filters/rSDSS.dat',
                           'filters/iSDSS.dat',
                           'filters/zSDSS.dat']

#making a grid of models with different redshifts
def exp_galaxy_grid(avs=np.array([0.0]), ages=np.array([0.0]), taus=np.array([0.0]), redshifts=np.array([0.0])):
    
    
    model_dict = {}
    model_dict['properties'] = []
    model_dict['model'] = []
    
    for properties in itertools.product(avs, ages, taus, redshifts):
        print('(av,age,tau,redshift)',properties)
   
        exp = {}                
        exp["age"] = properties[1]
        exp["tau"] = properties[2]          
        exp["massformed"] = 10.0
        exp["metallicity"] = 0.02
    
        dust = {}                    
        dust["type"] = "Calzetti"    
        dust["Av"] = properties[0]
        dust["eta"] = 2.
        
        nebular = {}
        nebular["logU"] = -3.
    
        model_components = {}  
        model_components["redshift"] = properties[3] 
        model_components["exponential"] = exp
        model_components["dust"] = dust
        model_components["nebular"] = nebular
        #model_components["veldisp"] = 100.0
        
        model = pipes.model_galaxy(model_components, filt_list=filter_list_splus)
        #print(model.spectrum_full[0])
        #print(model.redshifted_wavs)
        #fig = model.plot()
        model_dict['properties'].append(properties)
        model_dict['model'].append(model)
        #print(model_dict['properties'])
            
    pickle.dump(model_dict, file=open('z'+str(properties[3])+'_model_dict.h5', 'wb'))

ages = np.array([1, 3, 6, 10])
taus = np.array([1, 2, 3])
avs = np.array([0.])#, 0.2, 0.4, 0.6, 0.8, 1.])
redshifts = np.array([0.03])#, 0.002, 0.5])


exp_galaxy_grid(avs,ages,taus,redshifts)

(av,age,tau,redshift) (0.0, 1, 1, 0.03)
(av,age,tau,redshift) (0.0, 1, 2, 0.03)
(av,age,tau,redshift) (0.0, 1, 3, 0.03)
(av,age,tau,redshift) (0.0, 3, 1, 0.03)
(av,age,tau,redshift) (0.0, 3, 2, 0.03)
(av,age,tau,redshift) (0.0, 3, 3, 0.03)
(av,age,tau,redshift) (0.0, 6, 1, 0.03)
(av,age,tau,redshift) (0.0, 6, 2, 0.03)
(av,age,tau,redshift) (0.0, 6, 3, 0.03)
(av,age,tau,redshift) (0.0, 10, 1, 0.03)
(av,age,tau,redshift) (0.0, 10, 2, 0.03)
(av,age,tau,redshift) (0.0, 10, 3, 0.03)


In [ ]:
import bagpipes as pipes
import numpy as np
import itertools
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl

filter_list_splus = ['filters/F0378.dat',
                     'filters/F0395.dat',
                     'filters/F0410.dat',
                     'filters/F0430.dat',
                     'filters/F0515.dat',
                     'filters/F0660.dat',
                     'filters/F0861.dat',
                     'filters/uJAVA.dat',
                     'filters/gSDSS.dat',
                     'filters/rSDSS.dat',
                     'filters/iSDSS.dat',
                     'filters/zSDSS.dat']

filter_list_splus_broad = ['filters/uJAVA.dat',
                           'filters/gSDSS.dat',
                           'filters/rSDSS.dat',
                           'filters/iSDSS.dat',
                           'filters/zSDSS.dat']

#loading galaxy models
galaxy_list=pickle.load(file=open('z0.03_model_dict.h5', 'rb'))

#defining fit parameteres
exp = {}
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)                   
exp["massformed"] = (1., 15.)             
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = 0#(0., 3)
dust["eta"] = 2.

nebular = {}
nebular["logU"] = -3.

fit_instructions = {}                     
fit_instructions["redshift"] = 0# (0.,10.)
fit_instructions["exponential"] = exp
fit_instructions["dust"] = dust
fit_instructions["nebular"] = nebular

def galaxy_fit(i):
    #user-defined load_data function
    def load_data(id):
        if id == str(i):
            return np.array([galaxy_list['model'][i].photometry, 0.1 * galaxy_list['model'][i].photometry]).transpose()
    
    def load_data_broad(id):
        if id == str(i)+'_broad':
            return np.array([galaxy_list['model'][i].photometry, 0.1 * galaxy_list['model'][i].photometry]).transpose()[-5:]
    
   
                

    #making galaxy objects with models to create synthetic galaxies
    galaxy_to_fit = pipes.galaxy(str(i), load_data, filt_list=filter_list_splus, 
                                 phot_units=galaxy_list['model'][i].phot_units, spectrum_exists=False)
    galaxy_to_fit_broad = pipes.galaxy(str(i)+'_broad', load_data_broad, filt_list=filter_list_splus_broad, 
                                 phot_units=galaxy_list['model'][i].phot_units, spectrum_exists=False)

    #fig = galaxy_to_fit.plot()
    
    fit = pipes.fit(galaxy_to_fit, fit_instructions, run="exp_sfh")
    #fit_broad = pipes.fit(galaxy_to_fit_broad, fit_instructions, run="exp_sfh_broad")
    
    print('(av,age,tau,redshift)',galaxy_list['properties'][i])
    fit.fit(verbose=True)
    
    #print('(av,age,tau,redshift)',galaxy_list['properties'][i])
    #fit_broad.fit(verbose=True)
    
    fig = fit.plot_spectrum_posterior(save=True, show=True)
    fig = fit.plot_sfh_posterior(save=True, show=True)
    fig = fit.plot_corner(save=True, show=True)

    #fig = fit_broad.plot_spectrum_posterior(save=True, show=True)
    #fig = fit_broad.plot_sfh_posterior(save=True, show=True)
    #fig = fit_broad.plot_corner(save=True, show=True)
    
    ########################################
    
    fig = plt.figure(figsize=(12, 7))
    gs = mpl.gridspec.GridSpec(7, 4, hspace=3., wspace=0.1)

    ax1 = plt.subplot(gs[:4, :])

    pipes.plotting.add_observed_photometry(fit.galaxy, ax1, zorder=10)
    pipes.plotting.add_photometry_posterior(fit, ax1)

    labels = ["sfr", "mass_weighted_age", "stellar_mass", "ssfr"]
    
    post_quantities = dict(zip(labels, [fit.posterior.samples[l] for l in labels]))

    axes = []
    for i in range(4):
        axes.append(plt.subplot(gs[4:, i]))
        pipes.plotting.hist1d(post_quantities[labels[i]], axes[-1], smooth=True, label=labels[i])

    plt.show()
    #fig = galaxy_list['model'][i].plot()
    #fig = galaxy_list['model'][i].sfh.plot()
#for i in range(0,len(galaxy_list['properties'])):
galaxy_fit(1)
#my galaxies have metallicity 2.5 and massformed 10

#criar modelo com ruidos variados e ajustar - criar para redshift entre 0.002 e 0.5
#calcular redshift máximo para que halpha caia na banda de halfapha e para linha do o3, hBeta
#linhas de emissão/absorção - pesquisar


(av,age,tau,redshift) (0.0, 1, 2, 0.03)

Bagpipes: fitting object 1



In [ ]:
print(galaxy_list)